In [7]:
import os
import torch
import torchvision
from torchvision import transforms

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [8]:
from uetai.logger import WandbLogger, visualization_table

In [15]:
# Parameters
epochs = 5
batch_size = 32
lr = 0.005

# Init experiment
logger = WandbLogger(project_name='study_case_1')

# Prepare dataset

In [21]:
save_dir = './data'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))]
)

In [22]:
train_set = torchvision.datasets.CIFAR10(root=save_dir, train=True, download=True, transform=transform)
test_set  = torchvision.datasets.CIFAR10(root=save_dir, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [14]:
classes = ['plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [18]:
# Logging dataset artifact and visualization table

data_table = visualization_table(train_set)
logger.log_artifact(artifact_name='cifar-10', artifact_path='./data', vis_table=data_table)

wandb: Adding directory to artifact (./data)... Done. 0.3s


# Init model, optimizer, loss function

In [10]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Net().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer     = optim.Adam(model.parameters(), lr=lr)

# Training & Evaluate model

In [24]:
def train(model, trainloader, optimizer, loss_function, device, epoch):
    running_loss = 0

    for idx, (inputs, target) in enumerate(trainloader, 0):
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(inputs)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        logger.monitor(task='image_classify', 
                        inputs=inputs, 
                        ground_truths=target, 
                        output=output, 
                        epoch=epoch,
                        map2int=classes,
                        log_n_items=10
                    )


    total_loss = running_loss/len(trainloader.dataset)
    return total_loss

In [25]:
for epoch in range(epochs):
    train_loss = train(model, trainloader, optimizer, loss_function, device, epoch)
    print(f'Epoch {epoch+1} | Train loss: {train_loss}')

    logger.log({'train/loss': train_loss})

KeyboardInterrupt: 